In [1]:
n_iterations = 100_000

#collect_episodes_per_iteration = 5
replay_buffer_capacity = 2000

batch_size = 8
collect_steps_per_iteration = 1
lr_optimizer = 2.5e-4
log_interval = 5
n_eval_episodes = 10
eval_interval = 10
n_iterations_before_save = 5_000

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

#from tf_agents.environments import py_environment as pyenv, tf_py_environment, utils
#from tf_agents.specs import array_spec 
from tf_agents.trajectories import time_step
from tf_agents.environments.tf_py_environment import TFPyEnvironment
from tf_agents.networks.q_network import QNetwork
from tf_agents.agents.dqn.dqn_agent import DqnAgent
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.metrics import tf_metrics
from tf_agents.eval.metric_utils import log_metrics
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver
from tf_agents.drivers.dynamic_episode_driver import DynamicEpisodeDriver
from tf_agents.policies.random_tf_policy import RandomTFPolicy
from tf_agents.utils.common import function, Checkpointer
from tf_agents.trajectories import trajectory
from tf_agents.policies import random_tf_policy
from tf_agents.policies import policy_saver

import logging
import datetime

import os
import sys
sys.path.append(os.path.abspath(os.path.pardir))
import constants02
from constants02 import width, height, agent_state_id2str

from miner_env import MinerEnv
from tf_agents_miner_env import TFAgentsMiner

In [3]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import q_network
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

In [4]:
train_env = TFPyEnvironment(TFAgentsMiner())
eval_env = TFPyEnvironment(TFAgentsMiner())

Connected to server.
Found: map10
Found: map6
Found: map3
Found: map5
Found: map9
Found: map8
Found: map1
Found: map2
Found: map4
Found: map7
Connected to server.
Found: map10
Found: map6
Found: map3
Found: map5
Found: map9
Found: map8
Found: map1
Found: map2
Found: map4
Found: map7


In [5]:
## Creating the Deep Q-Network
preprocessing_layer = keras.layers.Lambda(
    lambda obs: tf.cast(obs, np.float32)/255.)
conv_layer_params = [(4,(3,3),1), (8,(3,3),1)]
fc_layer_params = [128, 64, 32]

q_net = QNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    conv_layer_params=conv_layer_params,
    fc_layer_params=fc_layer_params,
)

In [6]:
## Creating the DQN Agent
global_step = tf.compat.v1.train.get_or_create_global_step()
optimizer = keras.optimizers.RMSprop(lr=2.5e-4, rho=0.95, momentum=0.0, epsilon=0.00001, centered=True)

epsilon_fn = keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=1.0, # initial ε
    #decay_steps=380_000,
    decay_steps=30_000,
    end_learning_rate=0.01) # final ε

agent = DqnAgent(train_env.time_step_spec(),
                 train_env.action_spec(),
                 q_network=q_net,
                 optimizer=optimizer,
                 target_update_period=50,
                 td_errors_loss_fn=keras.losses.Huber(reduction="none"),
                 gamma=0.95, # discount factor
                 train_step_counter=global_step,
                 epsilon_greedy=lambda: epsilon_fn(global_step),
)

agent.initialize()

In [7]:
eval_policy = agent.policy
collect_policy = agent.collect_policy

In [8]:
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

In [9]:
example_environment = TFPyEnvironment(TFAgentsMiner())

Connected to server.
Found: map10
Found: map6
Found: map3
Found: map5
Found: map9
Found: map8
Found: map1
Found: map2
Found: map4
Found: map7


In [10]:
time_step = example_environment.reset()

In [11]:
random_policy.action(time_step)

PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([5], dtype=int32)>, state=(), info=())

## Metrics and Evaluation

The most common metric used to evaluate a policy is the average return. The return is the sum of rewards obtained while running a policy in an environment for an episode. Several episodes are run, creating an average return.

The following function computes the average return of a policy, given the policy, environment, and a number of episodes.


In [12]:
#@test {"skip": true}
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


# See also the metrics module for standard implementations of different metrics.
# https://github.com/tensorflow/agents/tree/master/tf_agents/metrics

Running this computation on the `random_policy` shows a baseline performance in the environment.

In [13]:
compute_avg_return(eval_env, random_policy, n_eval_episodes)

-1150.66

## Replay Buffer

The replay buffer keeps track of data collected from the environment. This tutorial uses `tf_agents.replay_buffers.tf_uniform_replay_buffer.TFUniformReplayBuffer`, as it is the most common. 

The constructor requires the specs for the data it will be collecting. This is available from the agent using the `collect_data_spec` method. The batch size and maximum buffer length are also required.


In [14]:
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_capacity)

For most agents, `collect_data_spec` is a named tuple called `Trajectory`, containing the specs for observations, actions, rewards, and other items.

## Data Collection

Now execute the random policy in the environment for a few steps, recording the data in the replay buffer.

In [15]:
#@test {"skip": true}
def collect_step(environment, policy, buffer):
  time_step = environment.current_time_step()
  action_step = policy.action(time_step)
  next_time_step = environment.step(action_step.action)
  traj = trajectory.from_transition(time_step, action_step, next_time_step)

  # Add trajectory to the replay buffer
  buffer.add_batch(traj)

def collect_data(env, policy, buffer, steps):
  for _ in range(steps):
    collect_step(env, policy, buffer)

collect_data(train_env, random_policy, replay_buffer, steps=100)

# This loop is so common in RL, that we provide standard implementations. 
# For more details see the drivers module.
# https://www.tensorflow.org/agents/api_docs/python/tf_agents/drivers

The replay buffer is now a collection of Trajectories.

In [16]:
# Dataset generates trajectories with shape [Bx2x...]
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, 
    sample_batch_size=batch_size, 
    num_steps=2).prefetch(3)


dataset

<PrefetchDataset shapes: (Trajectory(step_type=(8, 2), observation=(8, 2, 9, 21, 2), action=(8, 2), policy_info=(), next_step_type=(8, 2), reward=(8, 2), discount=(8, 2)), BufferInfo(ids=(8, 2), probabilities=(8,))), types: (Trajectory(step_type=tf.int32, observation=tf.float32, action=tf.int32, policy_info=(), next_step_type=tf.int32, reward=tf.float32, discount=tf.float32), BufferInfo(ids=tf.int64, probabilities=tf.float32))>

In [17]:
iterator = iter(dataset)

print(iterator)


In [18]:
next(iterator)

(Trajectory(step_type=<tf.Tensor: shape=(8, 2), dtype=int32, numpy=
 array([[1, 1],
        [1, 1],
        [1, 1],
        [1, 2],
        [1, 1],
        [1, 1],
        [1, 2],
        [1, 1]], dtype=int32)>, observation=<tf.Tensor: shape=(8, 2, 9, 21, 2), dtype=float32, numpy=
 array([[[[[ 2.00e+02,  0.00e+00],
           [-1.00e+01,  0.00e+00],
           [-1.00e+01,  0.00e+00],
           ...,
           [-5.10e+00,  0.00e+00],
           [ 4.50e+02,  0.00e+00],
           [-5.10e+00,  0.00e+00]],
 
          [[-1.00e+01,  0.00e+00],
           [-1.00e+01,  0.00e+00],
           [-2.00e+01,  0.00e+00],
           ...,
           [-1.00e+00,  0.00e+00],
           [-5.10e+00,  0.00e+00],
           [-1.00e+00,  0.00e+00]],
 
          [[-5.10e+00,  0.00e+00],
           [-1.00e+01,  0.00e+00],
           [-2.00e+01,  0.00e+00],
           ...,
           [ 1.50e+02,  0.00e+00],
           [-1.00e+00,  0.00e+00],
           [-1.00e+00,  5.00e+01]],
 
          ...,
 
          [[ 4

## Training the agent

Two things must happen during the training loop:

-   collect data from the environment
-   use that data to train the agent's neural network(s)

This example also periodicially evaluates the policy and prints the current score.

The following will take ~5 minutes to run.

In [19]:
now = datetime.datetime.now()
now_str = now.strftime("%Y%m%d-%H%M")
#script_name = __file__.split('.')[0]
script_name = "debug_empty_buffer"
save_dir = os.path.join("models", script_name)
os.makedirs(save_dir, exist_ok=True)
#logging.basicConfig(filename=os.path.join(save_path, f"log-{now_str}.txt"),level=logging.DEBUG)
checkpoint_dir = os.path.join(save_dir, 'checkpoint')
train_checkpointer = Checkpointer(
    ckpt_dir=checkpoint_dir,
    max_to_keep=500,
    agent=agent,
    policy=agent.policy,
    replay_buffer=replay_buffer,
    global_step=global_step
)

In [20]:
#@test {"skip": true}
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, n_eval_episodes)
returns = [avg_return]

for _ in range(n_iterations):

  # Collect a few steps using collect_policy and save to the replay buffer.
  for _ in range(collect_steps_per_iteration):
    collect_step(train_env, agent.collect_policy, replay_buffer)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience).loss

  step = agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0:7d}: loss = {1}'.format(step, train_loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, agent.policy, n_eval_episodes)
    print('step = {0:7d}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)
    if step > n_iterations_before_save:
      train_checkpointer.save(global_step)

step =       5: loss = 104.20884704589844
step =      10: loss = 167.33189392089844
step =      10: Average Return = -766.1099853515625
step =      15: loss = 35.94205093383789
step =      20: loss = 50.38216018676758
step =      20: Average Return = -1025.469970703125
step =      25: loss = 59.618499755859375
step =      30: loss = 120.24795532226562
step =      30: Average Return = -814.9000244140625
step =      35: loss = 17.072147369384766
step =      40: loss = 52.44712829589844
step =      40: Average Return = -4278.09033203125
step =      45: loss = 98.28364562988281
step =      50: loss = 121.4784927368164
step =      50: Average Return = -3427.150390625
step =      55: loss = 116.77301025390625
step =      60: loss = 273.4272155761719
step =      60: Average Return = -1424.159912109375
step =      65: loss = 116.74124908447266
step =      70: loss = 41.36418914794922
step =      70: Average Return = -984.1399536132812
step =      75: loss = 3.6193490028381348
step =      80: l

KeyboardInterrupt: 

## Visualization


### Plots

Use `matplotlib.pyplot` to chart how the policy improved during training.

One iteration of `Cartpole-v0` consists of 200 time steps. The environment gives a reward of `+1` for each step the pole stays up, so the maximum return for one episode is 200. The charts shows the return increasing towards that maximum each time it is evaluated during training. (It may be a little unstable and not increase monotonically each time.)

### Videos

Charts are nice. But more exciting is seeing an agent actually performing a task in an environment. 

First, create a function to embed videos in the notebook.

Now iterate through a few episodes of the Cartpole game with the agent. The underlying Python environment (the one "inside" the TensorFlow environment wrapper) provides a `render()` method, which outputs an image of the environment state. These can be collected into a video.

For fun, compare the trained agent (above) to an agent moving randomly. (It does not do as well.)